다음 데이터 세트를 이용하여 다이아몬드 가격예측(회귀) 

url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv" diamonds = pd.read_csv(url) 

1. ~ML (RF, DT, LR) 수행~

2. Dense layer만 이용 FNCC 구현

3. 순환 데이터 변환후 CNN 구현



각 단계별로 수행완료후

검사받은 이후에 다음 단계 진행

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv"
df = pd.read_csv(url)
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [3]:
print(np.unique(df['cut'], return_counts=True))
print(np.unique(df['color'], return_counts=True))
print(np.unique(df['clarity'], return_counts=True))

(array(['Fair', 'Good', 'Ideal', 'Premium', 'Very Good'], dtype=object), array([ 1610,  4906, 21551, 13791, 12082]))
(array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype=object), array([ 6775,  9797,  9542, 11292,  8304,  5422,  2808]))
(array(['I1', 'IF', 'SI1', 'SI2', 'VS1', 'VS2', 'VVS1', 'VVS2'],
      dtype=object), array([  741,  1790, 13065,  9194,  8171, 12258,  3655,  5066]))


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

origin_label_cut = df['cut'].copy()
origin_label_color = df['color'].copy()
origin_label_clarity = df['clarity'].copy()

df['cut'] = label_encoder.fit_transform(df['cut'])
df['color'] = label_encoder.fit_transform(df['color'])
df['clarity'] = label_encoder.fit_transform(df['clarity'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  int64  
 2   color    53940 non-null  int64  
 3   clarity  53940 non-null  int64  
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 4.1 MB


In [6]:
df.describe()

,carat,cut,color,clarity,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,2.553003,2.594197,3.835150,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.027708,1.701105,1.724591,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,0.000000,0.000000,0.000000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,2.000000,1.000000,2.000000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,2.000000,3.000000,4.000000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,3.000000,4.000000,5.000000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,4.000000,6.000000,7.000000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [7]:
len(df)

53940

# 1. Dense layer FCNN

In [8]:
import tensorflow as tf
from tensorflow import keras

2024-10-24 09:30:06.259042: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
X = df.drop('price', axis=1).to_numpy()
y = df['price'].to_numpy()

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.1,
                                                    shuffle=True,
                                                    random_state=42)
print(X_train.shape, y_train.shape)

(48546, 9) (48546,)


In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
# Create the model
model = keras.models.Sequential()

# Add layers
model.add(keras.layers.Dense(units=128,
                             activation='ReLU',
                             name='dense1',
                             input_shape=(9,)))
model.add(keras.layers.Dense(units=256,
                             activation='ReLU',
                             name='dense2'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(units=512,
                             activation='ReLU',
                             name='dense3'))
## regression에서는 output layer에 activate하지 않음
model.add(keras.layers.Dense(units=1,
                             name='output'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense1 (Dense)              (None, 128)               1280      
                                                                 
 dense2 (Dense)              (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense3 (Dense)              (None, 512)               131584    
                                                                 
 output (Dense)              (None, 1)                 513       
                                                                 
Total params: 166,401
Trainable params: 166,401
Non-trainable params: 0
_________________________________________________________________


2024-10-24 09:30:09.020328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
from keras.optimizers import Adam

model.compile(Adam(learning_rate=0.004), 
              loss='mean_squared_error', 
              metrics=['mean_squared_error'])

history = model.fit(X_train_scaled, y_train,
                    validation_split=0.1,
                    batch_size=512,
                    verbose=2,
                    epochs=30)

Epoch 1/30
86/86 - 1s - loss: 8837126.0000 - mean_squared_error: 8837126.0000 - val_loss: 1549434.3750 - val_mean_squared_error: 1549434.3750 - 865ms/epoch - 10ms/step
Epoch 2/30
86/86 - 0s - loss: 1715355.7500 - mean_squared_error: 1715355.7500 - val_loss: 1397897.0000 - val_mean_squared_error: 1397897.0000 - 312ms/epoch - 4ms/step
Epoch 3/30
86/86 - 0s - loss: 1553657.8750 - mean_squared_error: 1553657.8750 - val_loss: 1511034.2500 - val_mean_squared_error: 1511034.2500 - 323ms/epoch - 4ms/step
Epoch 4/30
86/86 - 0s - loss: 1425001.1250 - mean_squared_error: 1425001.1250 - val_loss: 1271253.3750 - val_mean_squared_error: 1271253.3750 - 317ms/epoch - 4ms/step
Epoch 5/30
86/86 - 0s - loss: 1266646.3750 - mean_squared_error: 1266646.3750 - val_loss: 1262487.8750 - val_mean_squared_error: 1262487.8750 - 287ms/epoch - 3ms/step
Epoch 6/30
86/86 - 0s - loss: 1204518.7500 - mean_squared_error: 1204518.7500 - val_loss: 1151345.1250 - val_mean_squared_error: 1151345.1250 - 341ms/epoch - 4ms/st

In [14]:
from sklearn.metrics import mean_squared_error

y_preds = model(X_test_scaled)
print(f"MSE: {mean_squared_error(y_preds, y_test)}")
# model.evaluate(X_test_scaled)

MSE: 491913.51913690026
